In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout
from tensorflow.keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define dataset paths
BASE_DIR = "/content/drive/MyDrive/Clothing Segmentation/dataset"
TRAIN_IMAGES_DIR = os.path.join(BASE_DIR, "train", "images")
TRAIN_MASKS_DIR = os.path.join(BASE_DIR, "train", "masks")
VAL_IMAGES_DIR = os.path.join(BASE_DIR, "val", "images")
VAL_MASKS_DIR = os.path.join(BASE_DIR, "val", "masks")

In [4]:
# Resize images for consistency
IMG_HEIGHT, IMG_WIDTH = 256, 256
NUM_CLASSES = 59  # Number of segmentation classes
BATCH_SIZE = 8

# Data augmentation
data_gen_args = dict(rotation_range=15,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     brightness_range=[0.8, 1.2],
                     horizontal_flip=True,
                     fill_mode='nearest')


In [19]:
class DataGenerator(Sequence):
    def __init__(self, image_dir, mask_dir, batch_size=BATCH_SIZE, img_size=(IMG_HEIGHT, IMG_WIDTH), num_classes=NUM_CLASSES, augment=False):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_classes = num_classes
        self.augment = augment

        self.image_filenames = sorted(os.listdir(image_dir))
        self.mask_filenames = sorted(os.listdir(mask_dir))

        self.image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args) if augment else None
        self.mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args) if augment else None

    def __len__(self):
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        batch_image_filenames = self.image_filenames[index * self.batch_size:(index + 1) * self.batch_size]
        batch_mask_filenames = self.mask_filenames[index * self.batch_size:(index + 1) * self.batch_size]

        images, masks = [], []

        for img_file, mask_file in zip(batch_image_filenames, batch_mask_filenames):
            # Load image
            img = load_img(os.path.join(self.image_dir, img_file), target_size=self.img_size)
            img = img_to_array(img) / 255.0  # Normalize
            images.append(img)

            # Load mask
            mask = load_img(os.path.join(self.mask_dir, mask_file), color_mode="grayscale", target_size=self.img_size)
            mask = img_to_array(mask).astype(np.uint8)
            mask = np.squeeze(mask)  # Convert to (H, W)
            mask = to_categorical(mask, num_classes=self.num_classes)  # One-hot encode
            masks.append(mask)

        images, masks = np.array(images), np.array(masks)

        # Apply data augmentation (if enabled)
        if self.augment:
            seed = np.random.randint(10000)  # Same seed for image & mask
            images = self.image_datagen.flow(images, batch_size=self.batch_size, seed=seed).__next__()


        return images, masks

In [20]:
train_generator = DataGenerator(TRAIN_IMAGES_DIR, TRAIN_MASKS_DIR, augment=True)
val_generator = DataGenerator(VAL_IMAGES_DIR, VAL_MASKS_DIR, augment=False)

In [21]:
print(f"Total training batches: {len(train_generator)}")
print(f"Total validation batches: {len(val_generator)}")

Total training batches: 87
Total validation batches: 18


In [22]:
def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES):
    inputs = Input(input_size)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs, outputs)
    return model


In [23]:
model = unet_model()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
# Callbacks: ModelCheckpoint, TensorBoard, ReduceLROnPlateau
checkpoint_cb = ModelCheckpoint("checkpoints/unet_best.h5", monitor="val_loss", save_best_only=True, verbose=1)
tensorboard_cb = TensorBoard(log_dir="logs")
reduce_lr_cb = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)

In [25]:
# Train the model with augmented data
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=20,
                    callbacks=[checkpoint_cb, tensorboard_cb, reduce_lr_cb])

# Save the final model
model.save("checkpoints/unet_full_model.h5")

Epoch 1/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7593 - loss: 3.1593
Epoch 1: val_loss improved from inf to 1.20934, saving model to checkpoints/unet_best.h5


87/87 ━━━━━━━━━━━━━━━━━━━━ 676s 7s/step - accuracy: 0.7593 - loss: 3.1485 - val_accuracy: 0.7682 - val_loss: 1.2093 - learning_rate: 1.0000e-04
Epoch 2/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 721ms/step - accuracy: 0.7666 - loss: 1.1274
Epoch 2: val_loss improved from 1.20934 to 1.03715, saving model to checkpoints/unet_best.h5


87/87 ━━━━━━━━━━━━━━━━━━━━ 88s 858ms/step - accuracy: 0.7666 - loss: 1.1271 - val_accuracy: 0.7682 - val_loss: 1.0371 - learning_rate: 1.0000e-04
Epoch 3/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 730ms/step - accuracy: 0.7672 - loss: 1.0479
Epoch 3: val_loss improved from 1.03715 to 1.02488, saving model to checkpoints/unet_best.h5


87/87 ━━━━━━━━━━━━━━━━━━━━ 83s 868ms/step - accuracy: 0.7671 - loss: 1.0478 - val_accuracy: 0.7682 - val_loss: 1.0249 - learning_rate: 1.0000e-04
Epoch 4/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 725ms/step - accuracy: 0.7655 - loss: 1.0105
Epoch 4: val_loss did not improve from 1.02488
87/87 ━━━━━━━━━━━━━━━━━━━━ 74s 848ms/step - accuracy: 0.7656 - loss: 1.0105 - val_accuracy: 0.7682 - val_loss: 1.0457 - learning_rate: 1.0000e-04
Epoch 5/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 759ms/step - accuracy: 0.7675 - loss: 1.0044
Epoch 5: val_loss improved from 1.02488 to 0.97993, saving model to checkpoints/unet_best.h5


87/87 ━━━━━━━━━━━━━━━━━━━━ 85s 881ms/step - accuracy: 0.7674 - loss: 1.0043 - val_accuracy: 0.7682 - val_loss: 0.9799 - learning_rate: 1.0000e-04
Epoch 6/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 710ms/step - accuracy: 0.7676 - loss: 1.0039
Epoch 6: val_loss did not improve from 0.97993
87/87 ━━━━━━━━━━━━━━━━━━━━ 73s 821ms/step - accuracy: 0.7675 - loss: 1.0038 - val_accuracy: 0.7682 - val_loss: 0.9853 - learning_rate: 1.0000e-04
Epoch 7/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 710ms/step - accuracy: 0.7674 - loss: 0.9862
Epoch 7: val_loss did not improve from 0.97993
87/87 ━━━━━━━━━━━━━━━━━━━━ 81s 816ms/step - accuracy: 0.7674 - loss: 0.9862 - val_accuracy: 0.7682 - val_loss: 0.9819 - learning_rate: 1.0000e-04
Epoch 8/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 717ms/step - accuracy: 0.7664 - loss: 0.9809
Epoch 8: val_loss improved from 0.97993 to 0.96728, saving model to checkpoints/unet_best.h5


...

87/87 ━━━━━━━━━━━━━━━━━━━━ 76s 870ms/step - accuracy: 0.7674 - loss: 0.8911 - val_accuracy: 0.7682 - val_loss: 0.8846 - learning_rate: 1.0000e-04
Epoch 14/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 713ms/step - accuracy: 0.7671 - loss: 0.8862
Epoch 14: val_loss improved from 0.88461 to 0.87057, saving model to checkpoints/unet_best.h5


87/87 ━━━━━━━━━━━━━━━━━━━━ 73s 835ms/step - accuracy: 0.7671 - loss: 0.8862 - val_accuracy: 0.7682 - val_loss: 0.8706 - learning_rate: 1.0000e-04
Epoch 15/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 721ms/step - accuracy: 0.7651 - loss: 0.8699
Epoch 15: val_loss did not improve from 0.87057
87/87 ━━━━━━━━━━━━━━━━━━━━ 74s 841ms/step - accuracy: 0.7651 - loss: 0.8698 - val_accuracy: 0.7684 - val_loss: 0.8737 - learning_rate: 1.0000e-04
Epoch 16/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 774ms/step - accuracy: 0.7671 - loss: 0.8464
Epoch 16: val_loss improved from 0.87057 to 0.85203, saving model to checkpoints/unet_best.h5


87/87 ━━━━━━━━━━━━━━━━━━━━ 82s 942ms/step - accuracy: 0.7671 - loss: 0.8465 - val_accuracy: 0.7684 - val_loss: 0.8520 - learning_rate: 1.0000e-04
Epoch 17/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 712ms/step - accuracy: 0.7681 - loss: 0.8419
Epoch 17: val_loss did not improve from 0.85203
87/87 ━━━━━━━━━━━━━━━━━━━━ 131s 816ms/step - accuracy: 0.7681 - loss: 0.8419 - val_accuracy: 0.7702 - val_loss: 0.8681 - learning_rate: 1.0000e-04
Epoch 18/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 710ms/step - accuracy: 0.7682 - loss: 0.8448
Epoch 18: val_loss improved from 0.85203 to 0.83359, saving model to checkpoints/unet_best.h5


87/87 ━━━━━━━━━━━━━━━━━━━━ 73s 838ms/step - accuracy: 0.7682 - loss: 0.8447 - val_accuracy: 0.7709 - val_loss: 0.8336 - learning_rate: 1.0000e-04
Epoch 19/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 700ms/step - accuracy: 0.7713 - loss: 0.8337
Epoch 19: val_loss did not improve from 0.83359
87/87 ━━━━━━━━━━━━━━━━━━━━ 80s 811ms/step - accuracy: 0.7713 - loss: 0.8337 - val_accuracy: 0.7738 - val_loss: 0.8346 - learning_rate: 1.0000e-04
Epoch 20/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 720ms/step - accuracy: 0.7725 - loss: 0.8158
Epoch 20: val_loss improved from 0.83359 to 0.81883, saving model to checkpoints/unet_best.h5


87/87 ━━━━━━━━━━━━━━━━━━━━ 74s 843ms/step - accuracy: 0.7725 - loss: 0.8160 - val_accuracy: 0.7708 - val_loss: 0.8188 - learning_rate: 1.0000e-04


In [27]:
import os

print("Current working directory:", os.getcwd())
print("Checkpoints directory exists:", os.path.exists("checkpoints"))
print("Files inside checkpoints directory:", os.listdir("checkpoints") if os.path.exists("checkpoints") else "Directory not found")


Current working directory: /content
Checkpoints directory exists: True
Files inside checkpoints directory: ['unet_best.h5', 'unet_full_model.h5']


In [28]:
import shutil

# Define source and destination paths
src_folder = "/content/checkpoints"
dst_folder = "/content/drive/MyDrive/checkpoints"

# Ensure the destination folder exists
os.makedirs(dst_folder, exist_ok=True)

# Move files
for file_name in os.listdir(src_folder):
    shutil.move(os.path.join(src_folder, file_name), os.path.join(dst_folder, file_name))

print("Checkpoints moved to Google Drive:", os.listdir(dst_folder))


Checkpoints moved to Google Drive: ['unet_best.h5', 'unet_full_model.h5']
